In [29]:
'''
this script takes as input the LSTM or RNN weights found by train.py
change the path in line 178 of this script to point to the h5 file
with LSTM or RNN weights generated by train.py
'''

from __future__ import division
from keras.models import load_model
import csv
import copy
import numpy as np
import distance
from jellyfish._jellyfish import damerau_levenshtein_distance
import unicodecsv
from sklearn import metrics
from math import sqrt
import time
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import Counter

In [32]:
from datetime import datetime
import time

#define the maximum length of events
def find_max_list(lst):
    list_len = [len(i) for i in lst]
    return max(list_len)


# define helper functions
def encode(sentence, times, times3, maxlen, act_indices, activities):
    print(sentence, times, times3)
    num_features = len(activities)+5
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        multiset_abstraction = Counter(sentence[:t+1])
        for c in activities:
            if c==char:
                X[0, t+leftpad, act_indices[c]] = 1
        X[0, t+leftpad, len(activities)] = t+1
        X[0, t+leftpad, len(activities)+1] = times[t]/divisor
        X[0, t+leftpad, len(activities)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(activities)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(activities)+4] = times3[t].weekday()/7
    return X

def getSymbol(predictions, target_indices_activities):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_activities[i]
        i += 1
    return symbol

eventlog = 'event_data_alexander_rounded.csv'
csvfile = open('data/%s' % eventlog, 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|')

#helper variables
lastcase = ''
firstLine = True
lineseq = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
caseidseqs = []
lines = []
caseids = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None

next(datareader, None)  # skip the headers
for row in datareader: #the columns are "CaseID,ActivityID,CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") #creates a datetime object from row[2]
    if row[0]!=lastcase:  #'lastcase' is to save the last executed case for the loop
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        if not firstLine:
            lineseq.append(lines)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
        lines = []
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds     #multiply with 60*60*24 = 86400 to go from days to seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds    #the .seconds method gives the time in seconds
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    lines.append(str(row[1])) #add the activity label to the line list
    caseids.append(row[0])
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    times4.append(timediff4)
    lasteventtime = t
    firstLine = False

# add last case
caseidseqs.append(caseids)
lineseq.append(lines)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))
divisor3 = np.mean(list(map(lambda x: np.mean(list(map(lambda y: x[len(x)-1]-y, x))), timeseqs2)))
print('divisor3: {}'.format(divisor3))
#########################################################################################################
# separate data into 3 parts
elems_per_fold = int(round(numlines/3))
fold1 = lineseq[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]
import csv
with open('code/output_files/folds/fold1.csv', 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold1, fold1_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])


fold2 = lineseq[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]
with open('code/output_files/folds/fold2.csv', 'w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold2, fold2_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])

fold3 = lineseq[2*elems_per_fold:]
fold3_c = caseids[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]
with open('code/output_files/folds/fold3.csv','w', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in zip(fold3, fold3_t):
        datawriter.writerow([f"{s}#{t}" for s, t in zip(row, timeseq)])

lines = fold3
caseids = fold3_c
lines_t = fold3_t
lines_t2 = fold3_t2
lines_t3 = fold3_t3

divisor: 8329.618428334255
divisor2: 257467.7155506364
divisor3: 288305.9640935422


In [38]:
len(lineseq)

174

In [39]:
len(caseids)

58

In [40]:
len(timeseqs)

174

In [36]:
print(len(fold3[0]))

41


In [37]:
print(len(fold3_c))

58


In [33]:
maxlen = find_max_list(fold1+fold2) #find maximum line size
# set parameters
predict_size = maxlen
activities = set([item for sublist in fold1+fold2 for item in sublist])
target_activities = copy.copy(activities)
print('there are:', len(activities), 'different activities')

print('total activities: {}, target activities: {}'.format(len(activities), len(target_activities)))
act_indices = dict((c, i) for i, c in enumerate(activities))
indices_act = dict((i, c) for i, c in enumerate(activities))
target_activities_indices = dict((c, i) for i, c in enumerate(target_activities))
target_indices_activities = dict((i, c) for i, c in enumerate(target_activities))
print(indices_act)

# load model, set this to the model generated by train.py
model = load_model('output_files/models/model_63-1.38.h5')

one_ahead_gt = []
one_ahead_pred = []

two_ahead_gt = []
two_ahead_pred = []

three_ahead_gt = []
three_ahead_pred = []

# make predictions
with open('output_files/results/suffix_and_remaining_time_%s' % eventlog, 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["CaseID", "Prefix length", "Groud truth", "Predicted", "Levenshtein", "Damerau", "Jaccard", "Ground truth times", "Predicted times", "RMSE", "MAE"])
    for prefix_size in range(2,maxlen):
        print(prefix_size)
        for line, caseid, times, times2, times3 in zip(lines, caseids, lines_t, lines_t2, lines_t3):
            times.append(0)
            cropped_line = line[:prefix_size]
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            if len(times2)<prefix_size:
                continue # make no prediction for this case, since this case has ended already
            ground_truth = ''.join(line[prefix_size:prefix_size+predict_size])
            ground_truth_t = times2[prefix_size-1]
            case_end_time = times2[len(times2)-1]
            ground_truth_t = case_end_time-ground_truth_t
            predicted = ''
            total_predicted_time = 0
            for i in range(predict_size):
                enc = encode(cropped_line, cropped_times, cropped_times3, maxlen, act_indices, activities)
                y = model.predict(enc, verbose=0) # make predictions
                # split predictions into seperate activity and time predictions
                y_char = y[0][0] 
                y_t = y[1][0][0]
                prediction = getSymbol(y_char, target_indices_activities) # undo one-hot encoding           
                cropped_line += prediction
                if y_t<0:
                    y_t=0
                cropped_times.append(y_t)
                if prediction == '!': # end of case was just predicted, therefore, stop predicting further into the future
                    one_ahead_pred.append(total_predicted_time)
                    one_ahead_gt.append(ground_truth_t)
                    print('! predicted, end case')
                    break
                y_t = y_t * divisor3
                cropped_times3.append(cropped_times3[-1] + timedelta(seconds=y_t))
                total_predicted_time = total_predicted_time + y_t
                predicted += prediction
            output = []
            if len(ground_truth)>0:
                output.append(caseid)
                output.append(prefix_size)
                output.append(str(ground_truth).encode("utf-8"))
                output.append(StopIteration(predicted))
                output.append(1 - distance.nlevenshtein(predicted, ground_truth))
                dls = 1 - (damerau_levenshtein_distance(str(predicted), str(ground_truth)) / max(len(predicted),len(ground_truth)))
                if dls<0:
                    dls=0 # we encountered problems with Damerau-Levenshtein Similarity on some linux machines where the default character encoding of the operating system caused it to be negative, this should never be the case
                output.append(dls)
                output.append(1 - distance.jaccard(predicted, ground_truth))
                output.append(ground_truth_t)
                output.append(total_predicted_time)
                output.append('')
                output.append(metrics.mean_absolute_error([ground_truth_t], [total_predicted_time]))
                #output.append(metrics.median_absolute_error([ground_truth_t], [total_predicted_time]))
                spamwriter.writerow(output)

there are: 15 different activities
total activities: 15, target activities: 15
{0: '13', 1: '8', 2: '15', 3: '9', 4: '14', 5: '2', 6: '10', 7: '5', 8: '12', 9: '4', 10: '11', 11: '1', 12: '6', 13: '3', 14: '7'}
2
['1', '10'] [0, 192] [datetime.datetime(2021, 10, 5, 6, 18, 58), datetime.datetime(2021, 10, 5, 6, 22, 10)]
['1', '10', '1', '0'] [0, 192, 0.13749832] [datetime.datetime(2021, 10, 5, 6, 18, 58), datetime.datetime(2021, 10, 5, 6, 22, 10), datetime.datetime(2021, 10, 5, 17, 22, 51, 585463)]


IndexError: list index out of range

In [ ]:
cropped_line, cropped_times, cropped_times3

In [ ]:
print('done')